In [ ]:
# Uploading CSV Files
#Source - https://kumbh.gov.in/Content/assets/Files/UtilituList/MKM25-TouristGuide.pdf
from google.colab import files
uploaded = files.upload()


Saving KUMBH_GUIDE.csv to KUMBH_GUIDE.csv


In [ ]:
#I have Converted PDF To CSV File and did some miinor changes

In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName('KumbhGuide').getOrCreate()

In [ ]:
df_pyspark=spark.read.csv('KUMBH_GUIDE.csv',header=True,inferSchema=True)
df_pyspark.show()

+---------+--------------------+--------------+----------------------+
|   Sr.No.|                Name|Contact Number|Knowledge of Languages|
+---------+--------------------+--------------+----------------------+
|        1|        Aarti Nishad|    9140423622|        Hindi, English|
|        2|   Aayushmann Mishra|    7307046461|        Hindi, English|
|        3|          Abha singh|    9628005212|  Hindi, English, B...|
|        4|   Abhay kumar Dixit|    7355992500|        Hindi, English|
|        5|  Abhay Kumar Maurya|    6307909120|        Hindi, English|
|        6|        Abhay Nishad|    9219601490|      Russian, Bengali|
|        7|       Abhijeet sahu|    8318032424|        Hindi, English|
|        8|     Abhimanyu kumar|    9616539117|               Marathi|
|        9| Abhinav Kumar Yadav|    7080980405|  Hindi, English, R...|
|       10|       Abhinav sarma|    8521309567|        Hindi, English|
|       11|        Abhineet vij|    7839316124|  Hindi, English, B...|
|     

In [ ]:
from pyspark.sql.functions import collect_set

# Get distinct values of 'Knowledge of Languages' column as a list
distinct_languages = df_pyspark.select(collect_set('Knowledge of Languages')).first()[0]

print(distinct_languages)

['Hindi, English, Spanish, Gujarati,', 'Hindi, English, French', 'Hindi, English, Tamil, French', 'Tamil, Telegu', 'Hindi, English, Spanish, Marwadi,', 'Hindi, English, Russian, Telegu', 'Hindi, English, Marathi, sanskrit', 'Hindi, Bengali,', 'Hindi, English, Russian, Tamil,', 'Hindi, English, Gujarati, Bhojpuri', 'Hindi, English, Russian, Kannad', 'Hindi, English, Gharwali', 'Hindi, Bengali, Punjabi', 'Hindi, English, Tamil, Gujarati,', 'Hindi, English, Bengali, Hindi,', 'Hindi, English, Spanish,', 'Hindi, German, Marathi, Bengali,', 'Marathi, French', 'Hindi,', 'Hindi, Telegu, Nepal', 'Russian, Tamil', 'Hindi, Bengali', 'Hindi, English, Tamil, Gujarati', 'Hindi, Samany English, Sanskrit', 'Hindi, English, Bengali, Bhojpuri', 'Hindi, English, German', 'Hindi, English, Gujarati, Bengali,', 'Hindi, English, Urdu', 'Hindi, English, German, Spanish,', 'Hindi, English, Russian, Marathi', 'Marathi', 'Gujarati, Bengali', 'Hindi, English, Russian, French', 'Hindi, English', 'Spanish, Telegu',

In [ ]:
from pyspark.sql.functions import trim

# Remove leading and trailing spaces from the 'Knowledge of Languages' column
df_trimmed = df_pyspark.withColumn('Knowledge of Languages', trim(df_pyspark['Knowledge of Languages']))

df_trimmed.show(truncate=False)

+---------+---------------------+--------------+---------------------------------+
|Sr.No.   |Name                 |Contact Number|Knowledge of Languages           |
+---------+---------------------+--------------+---------------------------------+
|1        |Aarti Nishad         |9140423622    |Hindi, English                   |
|2        |Aayushmann Mishra    |7307046461    |Hindi, English                   |
|3        |Abha singh           |9628005212    |Hindi, English, Bengali          |
|4        |Abhay kumar Dixit    |7355992500    |Hindi, English                   |
|5        |Abhay Kumar Maurya   |6307909120    |Hindi, English                   |
|6        |Abhay Nishad         |9219601490    |Russian, Bengali                 |
|7        |Abhijeet sahu        |8318032424    |Hindi, English                   |
|8        |Abhimanyu kumar      |9616539117    |Marathi                          |
|9        |Abhinav Kumar Yadav  |7080980405    |Hindi, English, Russian          |
|10 

In [ ]:
distinct_languages = df_trimmed.select(collect_set('Knowledge of Languages')).first()[0]
all_languages = [lang for sublist in distinct_languages for lang in sublist.split(',')]
unique_languages = list(set(all_languages))  # Get unique languages
print(unique_languages)

['', ' Kannad', 'Marathi', ' Gujarati', ' Bhojpuri', 'Bengali', 'German', ' Spanish', ' Arabic', ' Hindi', ' Chinese', 'Russian', ' Sanskrit', ' Avadhi', ' sanskrit', 'Telegu', ' Awadhi', ' Chhatisgarh', 'Japanese', ' English', ' Maithili', ' Tamil', 'Malayalam', 'French', ' Punjabi', 'Gujarati', ' Samany English', ' French', ' Japanese', ' Gharwali', ' Portuguese', ' Arabi', ' Marathi', 'Hindi', ' Bihar', ' Haryanvi', 'Tamil', ' Bengali', ' Marwadi', 'English', ' Russian', ' Bihari bhojpuri', ' Telegu', ' Bagheli', ' German', ' Urdu', 'Spanish', ' Nepal']


In [ ]:
from pyspark.sql.functions import regexp_replace

# Remove all extra spaces (including between words) from the 'Knowledge of Languages' column
df_trimmed = df_pyspark.withColumn('Knowledge of Languages', regexp_replace(df_pyspark['Knowledge of Languages'], r'\s+', ' '))

df_trimmed.show(truncate=False)

+---------+---------------------+--------------+---------------------------------+
|Sr.No.   |Name                 |Contact Number|Knowledge of Languages           |
+---------+---------------------+--------------+---------------------------------+
|1        |Aarti Nishad         |9140423622    |Hindi, English                   |
|2        |Aayushmann Mishra    |7307046461    |Hindi, English                   |
|3        |Abha singh           |9628005212    |Hindi, English, Bengali          |
|4        |Abhay kumar Dixit    |7355992500    |Hindi, English                   |
|5        |Abhay Kumar Maurya   |6307909120    |Hindi, English                   |
|6        |Abhay Nishad         |9219601490    |Russian, Bengali                 |
|7        |Abhijeet sahu        |8318032424    |Hindi, English                   |
|8        |Abhimanyu kumar      |9616539117    |Marathi                          |
|9        |Abhinav Kumar Yadav  |7080980405    |Hindi, English, Russian          |
|10 

In [ ]:
from pyspark.sql.functions import when, regexp_replace, lower, trim

# 1. Handle incomplete entries and potential extra characters (Row 12)
df_corrected = df_pyspark.withColumn(
    'Knowledge of Languages',
    when(df_pyspark['Knowledge of Languages'].contains('|'),
         regexp_replace(df_pyspark['Knowledge of Languages'], r'\|.*$', '')) # Remove everything after the pipe
    .otherwise(df_pyspark['Knowledge of Languages'])
)

# 2. Remove extra spaces and standardize case
df_corrected = df_corrected.withColumn(
    'Knowledge of Languages',
    lower(regexp_replace(trim(df_corrected['Knowledge of Languages']), r'\s*,\s*', ','))
)

# 3. Correct misspellings or variations
df_corrected = df_corrected.withColumn(
    'Knowledge of Languages',
    when(df_corrected['Knowledge of Languages'].contains('sanskrit'),
         regexp_replace(df_corrected['Knowledge of Languages'], 'sanskrit', 'sanskrit'))
    .when(df_corrected['Knowledge of Languages'].contains('arabi'),
         regexp_replace(df_corrected['Knowledge of Languages'], 'arabi', 'arabic'))
    .when(df_corrected['Knowledge of Languages'].contains('avadhi'),
         regexp_replace(df_corrected['Knowledge of Languages'], 'avadhi', 'awadhi'))
    .when(df_corrected['Knowledge of Languages'].contains('chhatisgarh'),
         regexp_replace(df_corrected['Knowledge of Languages'], 'chhatisgarh', 'chhatisgarhi'))
    .when(df_corrected['Knowledge of Languages'].contains('gharwali'),
         regexp_replace(df_corrected['Knowledge of Languages'], 'gharwali', 'garhwali'))
    .when(df_corrected['Knowledge of Languages'].contains('nepal'),
         regexp_replace(df_corrected['Knowledge of Languages'], 'nepal', 'nepali'))
    .when(df_corrected['Knowledge of Languages'].contains('punjabi'),
         regexp_replace(df_corrected['Knowledge of Languages'], 'punjabi', 'punjabi'))
    .when(df_corrected['Knowledge of Languages'].contains('samany english'),
         regexp_replace(df_corrected['Knowledge of Languages'], 'samany english', 'english'))
    .when(df_corrected['Knowledge of Languages'].contains('bhojpurii bhojpuri'),
         regexp_replace(df_corrected['Knowledge of Languages'], 'bhojpurii bhojpuri', 'bhojpuri'))
    .when(df_corrected['Knowledge of Languages'].contains('arabicc'),
         regexp_replace(df_corrected['Knowledge of Languages'], 'arabicc', 'arabic'))
    .when(df_corrected['Knowledge of Languages'].contains('bihar'),
         regexp_replace(df_corrected['Knowledge of Languages'], 'bihar', 'bhojpuri'))
    .otherwise(df_corrected['Knowledge of Languages'])
)


df_corrected.show(truncate=False)

+---------+---------------------+--------------+------------------------------+
|Sr.No.   |Name                 |Contact Number|Knowledge of Languages        |
+---------+---------------------+--------------+------------------------------+
|1        |Aarti Nishad         |9140423622    |hindi,english                 |
|2        |Aayushmann Mishra    |7307046461    |hindi,english                 |
|3        |Abha singh           |9628005212    |hindi,english,bengali         |
|4        |Abhay kumar Dixit    |7355992500    |hindi,english                 |
|5        |Abhay Kumar Maurya   |6307909120    |hindi,english                 |
|6        |Abhay Nishad         |9219601490    |russian,bengali               |
|7        |Abhijeet sahu        |8318032424    |hindi,english                 |
|8        |Abhimanyu kumar      |9616539117    |marathi                       |
|9        |Abhinav Kumar Yadav  |7080980405    |hindi,english,russian         |
|10       |Abhinav sarma        |8521309

In [ ]:
from pyspark.sql.functions import collect_set
distinct_languages4 = df_corrected.select(collect_set('Knowledge of Languages')).first()[0]
all_languages4 = [lang for sublist in distinct_languages4 for lang in sublist.split(',')]
unique_languages4 = list(set(all_languages4))  # Get unique languages
print(unique_languages4)

['', 'tamil', 'german', 'marwadi', 'telegu', 'punjabi', 'bhojpuri', 'hindi', 'kannad', 'bagheli', 'haryanvi', 'english', 'maithili', 'garhwali', 'gujarati', 'marathi', 'bengali', 'french', 'chinese', 'arabic', 'samany english', 'sanskrit', 'malayalam', 'spanish', 'japanese', 'nepali', 'russian', 'chhatisgarhi', 'urdu', 'bhojpurii bhojpuri', 'portuguese', 'awadhi', 'arabicc']


In [ ]:
df_corrected1 = df_corrected.withColumn(
    'Knowledge of Languages',
    when(df_corrected['Knowledge of Languages'].contains('samany english'),
         regexp_replace(df_corrected['Knowledge of Languages'], 'samany english', 'english'))
    .when(df_corrected['Knowledge of Languages'].contains('bhojpurii bhojpuri'),
         regexp_replace(df_corrected['Knowledge of Languages'], 'bhojpurii bhojpuri', 'bhojpuri'))
    .when(df_corrected['Knowledge of Languages'].contains('arabicc'),
         regexp_replace(df_corrected['Knowledge of Languages'], 'arabicc', 'arabic'))
    .when(df_corrected['Knowledge of Languages'].contains('bhojpurii bhojpuri'),
         regexp_replace(df_corrected['Knowledge of Languages'], 'bhojpurii bhojpuri', 'bhojpuri'))
    .otherwise(df_corrected['Knowledge of Languages'])
)


df_corrected1.show(truncate=False)

+---------+---------------------+--------------+------------------------------+
|Sr.No.   |Name                 |Contact Number|Knowledge of Languages        |
+---------+---------------------+--------------+------------------------------+
|1        |Aarti Nishad         |9140423622    |hindi,english                 |
|2        |Aayushmann Mishra    |7307046461    |hindi,english                 |
|3        |Abha singh           |9628005212    |hindi,english,bengali         |
|4        |Abhay kumar Dixit    |7355992500    |hindi,english                 |
|5        |Abhay Kumar Maurya   |6307909120    |hindi,english                 |
|6        |Abhay Nishad         |9219601490    |russian,bengali               |
|7        |Abhijeet sahu        |8318032424    |hindi,english                 |
|8        |Abhimanyu kumar      |9616539117    |marathi                       |
|9        |Abhinav Kumar Yadav  |7080980405    |hindi,english,russian         |
|10       |Abhinav sarma        |8521309

In [ ]:
from pyspark.sql.functions import collect_set
distinct_languages = df_corrected1.select(collect_set('Knowledge of Languages')).first()[0]
all_languages = [lang for sublist in distinct_languages for lang in sublist.split(',')]
unique_languages = list(set(all_languages))  # Get unique languages
print(unique_languages)

['', 'tamil', 'german', 'marwadi', 'telegu', 'bhojpuri', 'hindi', 'kannad', 'bagheli', 'haryanvi', 'english', 'maithili', 'garhwali', 'gujarati', 'marathi', 'bengali', 'french', 'chinese', 'arabic', 'sanskrit', 'malayalam', 'spanish', 'japanese', 'nepali', 'russian', 'chhatisgarhi', 'urdu', 'portuguese', 'awadhi', 'punjabi']


In [ ]:
from pyspark.sql.functions import col, isnan, when, count, isnull
from pyspark.sql.types import IntegerType
df_cleaned = df_corrected1.filter(col("`Sr.No.`").cast(IntegerType()).isNotNull())
df_cleaned.show()

+------+--------------------+--------------+----------------------+
|Sr.No.|                Name|Contact Number|Knowledge of Languages|
+------+--------------------+--------------+----------------------+
|     1|        Aarti Nishad|    9140423622|         hindi,english|
|     2|   Aayushmann Mishra|    7307046461|         hindi,english|
|     3|          Abha singh|    9628005212|  hindi,english,ben...|
|     4|   Abhay kumar Dixit|    7355992500|         hindi,english|
|     5|  Abhay Kumar Maurya|    6307909120|         hindi,english|
|     6|        Abhay Nishad|    9219601490|       russian,bengali|
|     7|       Abhijeet sahu|    8318032424|         hindi,english|
|     8|     Abhimanyu kumar|    9616539117|               marathi|
|     9| Abhinav Kumar Yadav|    7080980405|  hindi,english,rus...|
|    10|       Abhinav sarma|    8521309567|         hindi,english|
|    11|        Abhineet vij|    7839316124|  hindi,english,ben...|
|    12|    Abhishek Agrawal|    7398039855|  hi

In [ ]:
df_filled = df_cleaned.fillna(value="N/A", subset=['Knowledge of Languages', 'Contact Number'])

In [ ]:
df_filled.show(n=600, truncate=False)

+------+-----------------------+-----------------------------+-------------------------------------------------------+
|Sr.No.|Name                   |Contact Number               |Knowledge of Languages                                 |
+------+-----------------------+-----------------------------+-------------------------------------------------------+
|1     |Aarti Nishad           |9140423622                   |hindi,english                                          |
|2     |Aayushmann Mishra      |7307046461                   |hindi,english                                          |
|3     |Abha singh             |9628005212                   |hindi,english,bengali                                  |
|4     |Abhay kumar Dixit      |7355992500                   |hindi,english                                          |
|5     |Abhay Kumar Maurya     |6307909120                   |hindi,english                                          |
|6     |Abhay Nishad           |9219601490      

In [ ]:
from pyspark.sql.functions import length, when

# 1. Change contact numbers with length greater than 10
df_filled = df_filled.withColumn(
    "Contact Number",
    when(length(df_filled["Contact Number"]) > 10, "Invalid Number").otherwise(df_filled["Contact Number"])
)

# 2. Fill NaN values with "N/A"
df_filled = df_filled.fillna(value="N/A", subset=["Contact Number"])

In [ ]:
df_filled.show(n=600, truncate=False)

+------+-----------------------+--------------+-------------------------------------------------------+
|Sr.No.|Name                   |Contact Number|Knowledge of Languages                                 |
+------+-----------------------+--------------+-------------------------------------------------------+
|1     |Aarti Nishad           |9140423622    |hindi,english                                          |
|2     |Aayushmann Mishra      |7307046461    |hindi,english                                          |
|3     |Abha singh             |9628005212    |hindi,english,bengali                                  |
|4     |Abhay kumar Dixit      |7355992500    |hindi,english                                          |
|5     |Abhay Kumar Maurya     |6307909120    |hindi,english                                          |
|6     |Abhay Nishad           |9219601490    |russian,bengali                                        |
|7     |Abhijeet sahu          |8318032424    |hindi,english    

In [ ]:
from pyspark.sql.functions import split, array_contains, when, lit

# 1. Get unique languages
unique_languages = df_filled.select("Knowledge of Languages").distinct().rdd.flatMap(lambda x: x[0].split(",")).distinct().collect()

# 2. Create new columns for each language
for language in unique_languages:
    df_filled = df_filled.withColumn(
        language,
        when(array_contains(split(df_filled["Knowledge of Languages"], ","), language), language).otherwise(lit(None))
    )


In [ ]:
df_filled.show(n=600, truncate=False)

+------+-----------------------+--------------+-------------------------------------------------------+-----+-------+-------+------+----+------+--------+-------+-------+------+------+--------+--------+-----+-------+--------+-------+------+--------+--------+---------+--------+------------+------+----+------+-------+----------+----+-------+-------+
|Sr.No.|Name                   |Contact Number|Knowledge of Languages                                 |hindi|english|russian|telegu|    |german|japanese|spanish|marathi|french|awadhi|haryanvi|maithili|tamil|punjabi|sanskrit|bengali|kannad|gujarati|bhojpuri|malayalam|garhwali|chhatisgarhi|arabic|urdu|nepali|chinese|portuguese|N/A |marwadi|bagheli|
+------+-----------------------+--------------+-------------------------------------------------------+-----+-------+-------+------+----+------+--------+-------+-------+------+------+--------+--------+-----+-------+--------+-------+------+--------+--------+---------+--------+------------+------+----+-

In [ ]:
from pyspark.sql.functions import split, array_contains, when, lit

# 1. Get unique languages
unique_languages = df_filled.select("Knowledge of Languages").distinct().rdd.flatMap(lambda x: x[0].split(",")).distinct().collect()

# 2. Create new columns for each language with "yes" or "no"
for language in unique_languages:
    df_filled = df_filled.withColumn(
        language,
        when(array_contains(split(df_filled["Knowledge of Languages"], ","), language), "yes").otherwise("no")
    )

In [ ]:
df_filled.show(n=600, truncate=False)

+------+-----------------------+--------------+-------------------------------------------------------+-----+-------+-------+------+---+------+--------+-------+-------+------+------+--------+--------+-----+-------+--------+-------+------+--------+--------+---------+--------+------------+------+----+------+-------+----------+---+-------+-------+
|Sr.No.|Name                   |Contact Number|Knowledge of Languages                                 |hindi|english|russian|telegu|   |german|japanese|spanish|marathi|french|awadhi|haryanvi|maithili|tamil|punjabi|sanskrit|bengali|kannad|gujarati|bhojpuri|malayalam|garhwali|chhatisgarhi|arabic|urdu|nepali|chinese|portuguese|N/A|marwadi|bagheli|
+------+-----------------------+--------------+-------------------------------------------------------+-----+-------+-------+------+---+------+--------+-------+-------+------+------+--------+--------+-----+-------+--------+-------+------+--------+--------+---------+--------+------------+------+----+------